In [18]:
from keras.models import Sequential
from scipy.misc import imread
get_ipython().magic('matplotlib inline')
import matplotlib.pyplot as plt
import numpy as np
import keras
from keras.layers import Dense
import pandas as pd

from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np
from keras.applications.vgg16 import decode_predictions
from keras.utils.np_utils import to_categorical

from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.optimizers import SGD
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Dropout, Flatten, merge, Reshape, Activation

from sklearn.metrics import log_loss
import os

In [3]:
labels_path = os.getcwd() + "\\" + os.path.join( "data", "labels.csv", "")

In [4]:
labels_path

'C:\\Users\\Vladamir Mazin\\DeepNN\\data\\labels.csv\\'

In [5]:
data_labels = pd.read_csv(labels_path + "labels.csv")

In [6]:
type(data_labels)

pandas.core.frame.DataFrame

In [7]:
df = pd.DataFrame(np.random.randn(100, 2))
# df

In [8]:
# def get_train_test(data_labels):
#     msk = np.random.rand(len(data_labels)) < 0.8
#     train = data_labels[msk]
#     test = data_labels[~msk]
#     return train, test

In [9]:
# train, test = get_train_test(data_labels)

In [10]:
# train['id']

In [11]:
img_path = os.getcwd() + "\\" + os.path.join( "data", "train", "train", "")

In [12]:
# from itertools import islice
# for x in islice(train.iterrows(), 10):
#     print(x[1]['id'])

In [13]:
# # preparing the train dataset
# train_img = []
# for row in train.iterrows():
#     temp_img = image.load_img(img_path + row[1]['id'] + '.jpg', target_size=(224,224))
#     temp_img = image.img_to_array(temp_img)
#     train_img.append(temp_img)
    
# #converting train images to array and applying mean subtraction processing
# train_img = np.array(train_img) 
# train_img = preprocess_input(train_img)

In [14]:
# # applying the same procedure with the test dataset
# test_img=[]
# for row in test.iterrows():
#     temp_img = image.load_img(img_path + row[1]['id'] + '.jpg', target_size=(224,224))
#     temp_img = image.img_to_array(temp_img)
#     test_img.append(temp_img)

# test_img=np.array(test_img) 
# test_img=preprocess_input(test_img)

#### Let's consider all the data:

In [30]:
train = data_labels

In [34]:
# applying the same procedure with the test dataset
# train_img = []
# cnt = 1
# for row in train.iterrows():
#     temp_img = image.load_img(img_path + row[1]['id'] + '.jpg', target_size=(224,224))
#     temp_img = image.img_to_array(temp_img)
#     train_img.append(temp_img)
#     if cnt % 100 == 0:
#         print("{} images processed".format(cnt))
#     cnt += 1
# train_img = np.array(train_img) 
# train_img = preprocess_input(train_img)

In [33]:
train_img.shape

(10222, 224, 224, 3)

In [35]:
from keras.models import Model

def vgg16_model(img_rows, img_cols, channel=1, num_classes=None):

    model = VGG16(weights='imagenet', include_top=True)

    model.layers.pop()

    model.outputs = [model.layers[-1].output]

    model.layers[-1].outbound_nodes = []

    x=Dense(num_classes, activation='softmax')(model.output)

    model=Model(model.input,x)

    #To set the first 8 layers to non-trainable (weights will not be updated)
    for layer in model.layers[:8]:
        layer.trainable = False

    # Learning rate is changed to 0.001
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [36]:
train_y = np.asarray(train['breed'])

le = LabelEncoder()

train_y = le.fit_transform(train_y)

train_y=to_categorical(train_y)

train_y=np.array(train_y)

In [37]:
from sklearn.model_selection import train_test_split
X_train, X_valid, Y_train, Y_valid=train_test_split(train_img,train_y,test_size=0.2, random_state=42)

In [40]:
X_valid.shape

(2045, 224, 224, 3)

In [41]:
# Example to fine-tune on 3000 samples from Cifar10
img_rows, img_cols = 224, 224 # Resolution of inputs
channel = 3
num_classes = 120 
batch_size = 32
nb_epoch = 10

In [42]:
# Load our model
model = vgg16_model(img_rows, img_cols, channel, num_classes)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [46]:
# Start Fine-tuning
model.fit(X_train, Y_train,batch_size=batch_size,epochs=nb_epoch,shuffle=True,verbose=1,validation_data=(X_valid, Y_valid))

Train on 8177 samples, validate on 2045 samples
Epoch 1/10
8177/8177 [==============================] - 172s 21ms/step - loss: 4.7779 - acc: 0.0353 - val_loss: 4.7672 - val_acc: 0.0826
Epoch 2/10
8177/8177 [==============================] - 149s 18ms/step - loss: 4.7569 - acc: 0.1419 - val_loss: 4.7520 - val_acc: 0.1658
Epoch 3/10
8177/8177 [==============================] - 148s 18ms/step - loss: 4.7415 - acc: 0.2059 - val_loss: 4.7398 - val_acc: 0.2166
Epoch 4/10
8177/8177 [==============================] - 149s 18ms/step - loss: 4.7268 - acc: 0.2637 - val_loss: 4.7260 - val_acc: 0.2724
Epoch 5/10
8177/8177 [==============================] - 148s 18ms/step - loss: 4.7112 - acc: 0.3233 - val_loss: 4.7130 - val_acc: 0.3007
Epoch 6/10
8177/8177 [==============================] - 148s 18ms/step - loss: 4.6960 - acc: 0.3477 - val_loss: 4.7011 - val_acc: 0.3178
Epoch 7/10
8177/8177 [==============================] - 148s 18ms/step - loss: 4.6806 - acc: 0.3686 - val_loss: 4.6881 - val_acc: 

In [43]:
# Make predictions
predictions_valid = model.predict(X_valid, batch_size=batch_size, verbose=1)

2045/2045 [==============================] - 17s 8ms/step


In [44]:
# Cross-entropy loss score
score = log_loss(Y_valid, predictions_valid)

In [45]:
score

4.7878120063278082